In [15]:
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score


def get_data(filename = "data/", rewards = False, humanDict = {'"beginner_non_tired_collaborative.POMDPx"': 0,
 '"beginner_non_tired_non_collaborative.POMDPx"': 1,
 '"beginner_tired_collaborative.POMDPx"': 2,
 '"beginner_tired_non_collaborative.POMDPx"': 3,
 '"distracted_collaborative.POMDPx"': 4,
 '"distracted_non_collaborative.POMDPx"': 5,
 '"expert_non_tired_collaborative.POMDPx"': 6,
 '"expert_non_tired_non_collaborative.POMDPx"': 7,
 '"expert_tired_collaborative.POMDPx"': 8,
 '"expert_tired_non_collaborative.POMDPx"': 9,
 '"humanModel_v2.POMDPx"': 10}):
    #data = pd.read_csv(filename)

# 	start = data.Status=='START'
# 	data['TaskNr'] = start.cumsum()
# 	mean = data.groupby(by=['TaskNr']).agg("mean")
# 	start = data[start]
# 	start[['robotmodel','humanmodel']]
# 	mean['robotmodel']=start['robotmodel'].values
# 	mean['humanmodel']=start['humanmodel'].values

    

    raw = pd.read_csv(filename +"raw_data.csv")
    obs =pd.read_csv(filename + "human_observables.csv")
    models = raw[raw.human_type!="''"][['task_id','human_type','robot_model']]
    data = pd.merge(models, obs, on='task_id', how='left')
    mean = data.groupby('task_id').mean()
    mean = pd.merge(mean, models, on='task_id', how='left')
    
    #extract datapoints and labels
    observations = mean.values[:,5:11]
    if(rewards):
        observations = np.hstack((mean.values[:,5:11], mean.values[:, -3][:,None]))

    observations = np.array(observations, dtype=np.float64)
    nan_mask = np.ones(observations.shape[0], dtype=bool)
    nan_mask[np.unique(np.argwhere((np.isnan(observations) * 1))[:,0])] = False
    observations = observations[nan_mask]

    labels = mean.values[:,-2][nan_mask]
    human_types = sorted(np.unique(labels))

    #here classes are assigned
    for i in range(len(labels)):
        labels[i] = humanDict[labels[i]]

    return observations, np.array(labels, dtype = np.int), mean, human_types

def train(X, Y):
    # usps = scipy.io.loadmat('data/usps.mat')
    # X = usps['data_patterns'].T[:200]
    # Y = np.argmax(usps['data_labels'], axis = 0)[:200]

    n, d = X.shape

    Cs = np.logspace(-2,2,10)
    gammas = np.logspace(-2,2,10)

    svc = svm.SVC(kernel='rbf', decision_function_shape='ovo', probability = True)
    clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs, gamma = gammas),n_jobs=-1, cv = 5)
    clf.fit(X, Y)
    print(clf.predict_proba([X[0]]))        

    print('best score %f'%clf.best_score_)
    return clf

In [16]:
import pickle
X,Y,_,_ = get_data("hrc_industry_ss18/code/results/daten2/")
svm = train(X,Y)
binary_file = open('svm.bin',mode='wb')
pickle.dump(svm, binary_file)
binary_file.close()

[[1.04724677e-02 1.16635808e-04 3.35946929e-04 4.48422798e-04
  4.89014453e-02 5.94122604e-02 1.21426908e-01 1.13516311e-01
  1.32589967e-01 2.76431234e-01 2.36348401e-01]]
best score 0.446770


In [13]:
bytes_read = open("svm.bin", "rb").read()
svm = pickle.loads(bytes_read)
bytes_read = open("robotDictInv.bin", "rb").read()
robotDictInv = pickle.loads(bytes_read)
svm.predict_proba(np.zeros(6).reshape(1,6))

array([[0.00316906, 0.15741541, 0.37064651, 0.40530442, 0.00165792,
        0.00226531, 0.00651991, 0.00613414, 0.01073854, 0.03367775,
        0.00247104]])

In [8]:
raw = pd.read_csv("/home/simon/hrc_industry_ss18/code/results/daten/raw_data.csv")
obs =pd.read_csv("/home/simon/hrc_industry_ss18/code/results/daten/human_observables.csv")
models = raw[raw.human_type!="''"][['task_id','human_type','robot_model']]
data = pd.merge(models, obs, on='task_id', how='left')
mean = data.groupby('task_id').mean()
mean = pd.merge(mean, models, on='task_id', how='left')
mean

,task_id,step_count,update_received_time,update_secs,update_nsecs,Human Detected,Looked Around,Succesfull Grasp,Failed Grasp,Warned Robot,Stood Idle,human_type,robot_model
0,2,1.5,NaN,1.533210e+09,2.853494e+08,1.000000,0.000000,0.500000,0.000000,0.000000,0.500000,"""expert_tired_non_collaborative.POMDPx""","""proactive_robot_pomdp_beginner.pomdpx"""
1,3,1.0,NaN,1.533210e+09,8.800561e+08,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,"""expert_tired_non_collaborative.POMDPx""","""proactive_robot_pomdp_beginner.pomdpx"""
2,4,2.5,NaN,1.533210e+09,2.053032e+08,1.000000,0.000000,0.000000,0.750000,0.000000,0.250000,"""expert_tired_non_collaborative.POMDPx""","""proactive_robot_pomdp_beginner.pomdpx"""
3,5,1.5,NaN,1.533210e+09,3.247584e+08,1.000000,0.000000,0.500000,0.000000,0.000000,0.500000,"""expert_tired_non_collaborative.POMDPx""","""proactive_robot_pomdp_beginner.pomdpx"""
4,6,1.5,NaN,1.533210e+09,2.888734e+08,1.000000,0.000000,0.500000,0.000000,0.000000,0.500000,"""expert_tired_non_collaborative.POMDPx""","""proactive_robot_pomdp_beginner.pomdpx"""
5,7,1.0,NaN,1.533210e+09,7.558286e+08,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,"""expert_tired_non_collaborative.POMDPx""","""proactive_robot_pomdp_beginner.pomdpx"""
6,8,1.5,NaN,1.533210e+09,9.326158e+08,1.000000,0.000000,0.500000,0.000000,0.500000,0.000000,"""expert_tired_non_collaborative.POMDPx""","""proactive_robot_pomdp_beginner.pomdpx"""
7,9,3.5,NaN,1.533210e+09,3.948372e+08,1.000000,0.000000,0.000000,0.500000,0.333333,0.166667,"""expert_tired_non_collaborative.POMDPx""","""proactive_robot_pomdp_beginner.pomdpx"""
8,10,1.5,NaN,1.533210e+09,5.402769e+08,1.000000,0.000000,0.500000,0.000000,0.000000,0.500000,"""expert_tired_non_collaborative.POMDPx""","""proactive_robot_pomdp_beginner.pomdpx"""
9,11,1.0,NaN,1.533210e+09,9.701164e+08,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,"""expert_tired_non_collaborative.POMDPx""","""proactive_robot_pomdp_beginner.pomdpx"""


In [90]:
from collections import deque
context = deque([None]*10)
observations = np.zeros(6)
context.append(observations)
context = deque([observations]*10)
np.mean(context,axis=0)

array([0., 0., 0., 0., 0., 0.])